<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RecSys/blob/main/Paul%20Himyak/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import re
import numpy as np
import seaborn as sns
from datetime import datetime

from tensorflow.keras.preprocessing.text import Tokenizer

import gdown

In [44]:
dataset_url = "https://docs.google.com/spreadsheets/d/1ErXvN290PPthLGsb0ALpa_0aUDnvC0jg/edit?usp=share_link&ouid=113369956649761615778&rtpof=true&sd=true"
dataset_filename = "profpatolog_modified.xlsx"
#!wget -qc dataset_url -O dataset_filename
gdown.download(dataset_url, output=dataset_filename, quiet=True )


/usr/local/lib/python3.9/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(


'profpatolog_modified.xlsx'

In [47]:
from google.colab import drive

# Монтируем Google Диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
df = pd.read_excel("/content/drive/MyDrive/profpatolog_modified.xlsx")


In [50]:
#Удаляем случайные вбросы
df.drop([843, 851, 1022, 1024], inplace = True)

#Убираем пустые колонки
empty_columns = df.columns[df.isna().all()]
df.drop(empty_columns, inplace = True, axis = 1)

In [51]:
FIELD_CLIENT_AGE = "КлиентВозраст"
FIELD_CLIENT_BIRTH_DAY = "КлиентДатаРождения"
FIELD_CLIENT_GENDER = "КлиентПол"
FIELD_CLIENT_GENDER_M = 0
FIELD_CLIENT_GENDER_F = 1
FIELD_CONCLUSION = "ЗаключениеМК"
FIELD_CONCLUSION_FIT = 0
FIELD_CONCLUSION_PART_FIT = 1
FIELD_CONCLUSION_NOT_FIT = 2

FIELD_DISPANSARY_GROUP = "ДиспансернаяГруппа"
FIELD_DISPANSARY_GROUP_I = 0
FIELD_DISPANSARY_GROUP_II = 1
FIELD_DISPANSARY_GROUP_III = 2
FIELD_DISPANSARY_GROUP_OTHER = 3

FIELD_HARMFUL_FACTORS = "ВредныеФакторы"

In [53]:
mkb_columns = df.filter(regex="(?i)МКБ10").columns


In [59]:
df1 = df[FIELD_HARMFUL_FACTORS].str.split(",")

In [63]:
harmful_values = df1.explode().sort_values().apply(lambda x: x.replace(".", "x")).unique()

In [65]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='unknown', char_level=False)
tokenizer.fit_on_texts(harmful_values)

In [68]:
tokenizer.

dict_items([('unknown', 1), ('1x1', 2), ('1x14x1', 3), ('1x14x2', 4), ('1x15', 5), ('1x17', 6), ('1x18x3', 7), ('1x19x1', 8), ('1x2', 9), ('1x23', 10), ('1x24', 11), ('1x29', 12), ('1x29x1', 13), ('1x29x2', 14), ('1x29x4', 15), ('1x3', 16), ('1x30x1', 17), ('1x34x1', 18), ('1x36', 19), ('1x36x1', 20), ('1x37', 21), ('1x37x1', 22), ('1x37x1x1', 23), ('1x37x1x2', 24), ('1x39', 25), ('1x45x1', 26), ('1x46', 27), ('1x48', 28), ('1x49', 29), ('1x49x14', 30), ('1x49x2', 31), ('1x50', 32), ('1x8x1', 33), ('1x8x1x1', 34), ('1x8x1x2', 35), ('1x8x2', 36), ('1x8x2x1', 37), ('1x8x2x2', 38), ('1x8x3', 39), ('1x8x3x1', 40), ('1x8x3x2', 41), ('10', 42), ('11', 43), ('11x1', 44), ('11x2', 45), ('11x3', 46), ('11x4', 47), ('12', 48), ('13', 49), ('14', 50), ('15', 51), ('17', 52), ('18', 53), ('18x1', 54), ('18x2', 55), ('2x4', 56), ('2x4x3', 57), ('22', 58), ('23', 59), ('24', 60), ('25', 61), ('26', 62), ('27', 63), ('3x1', 64), ('3x1x2', 65), ('3x1x7', 66), ('3x1x8x1', 67), ('3x1x8x2', 68), ('3x1x8x